# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster_response', engine)

In [4]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [6]:
df.shape

(26216, 40)

In [7]:
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [8]:
X[:10]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
5               Information about the National Palace-
6                       Storm at sacred heart of jesus
7    Please, we need tents and water. We are in Sil...
8      I would like to receive the messages, thank you
9    I am in Croix-des-Bouquets. We have health iss...
Name: message, dtype: object

In [9]:
y[:10]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
X.shape

(26216,)

In [11]:
y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    '''
    Tokenization function to process data.
    
    Parameters:
    text (pandas Series): text to be processed
    
    Returns:
    clean_tokens (list): list of words converted to a lower case with removed stop words
    '''
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    clean_tokens = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 777)

model_initial = pipeline.fit(X_train, y_train)

CPU times: user 1min 27s, sys: 443 ms, total: 1min 28s
Wall time: 1min 28s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
def eval_metrics(real_labels, predicted_labels, col_names):
    '''
    Evalute ML pipeline model's performance.
    
    Parameters:
    real_labels (list): real labels
    predicted_labels (list): predicted labels
    col_names (list): list of strings containing column names.
       
    Returns:
    performance_metrics (pandas DataFrame): accuracy, precision, recall and f1 score
    '''
    
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(real_labels[:, i], predicted_labels[:, i])
        precision = precision_score(real_labels[:, i], predicted_labels[:, i], average='micro')
        recall = recall_score(real_labels[:, i], predicted_labels[:, i], average='micro')
        f1 = f1_score(real_labels[:, i], predicted_labels[:, i], average='micro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    performance_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return performance_metrics

In [16]:
%%time
# Calculate evaluation metrics for test set
y_test_pred = pipeline.predict(X_test)
col_names = list(y.columns.values)

eval_metrics_test = eval_metrics(np.array(y_test), y_test_pred, col_names)

CPU times: user 9.5 s, sys: 152 ms, total: 9.65 s
Wall time: 9.77 s


In [17]:
eval_metrics_test

,Accuracy,Precision,Recall,F1
related,0.809735,0.809735,0.809735,0.809735
request,0.878700,0.878700,0.878700,0.878700
offer,0.994660,0.994660,0.994660,0.994660
aid_related,0.745957,0.745957,0.745957,0.745957
medical_help,0.919439,0.919439,0.919439,0.919439
medical_products,0.948123,0.948123,0.948123,0.948123
search_and_rescue,0.971315,0.971315,0.971315,0.971315
security,0.981385,0.981385,0.981385,0.981385
military,0.967653,0.967653,0.967653,0.967653
child_alone,1.000000,1.000000,1.000000,1.000000


In [18]:
eval_metrics_test.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.942558,0.942558,0.942558,0.942558
std,0.057696,0.057696,0.057696,0.057696
min,0.745957,0.745957,0.745957,0.745957
25%,0.930310,0.930310,0.930310,0.930310
50%,0.954913,0.954913,0.954913,0.954913
75%,0.983140,0.983140,0.983140,0.983140
max,1.000000,1.000000,1.000000,1.000000


In [19]:
# Print mean f1 score
print("F1 score mean: ", eval_metrics_test['F1'].mean())

F1 score mean:  0.942558403689


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
# Read current parameters:
print(pipeline.get_params())

{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f7832f0ff28>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 

In [21]:
# Define mean f1 score to use in grid search scoring object
def define_mean_f1_score(real_labels, predicted_labels):
    """Mean F1 score for all classifiers.
    
    Parameters:
    real_labels (list): real labels
    predicted_labels (list): predicted labels
        
    Returns:
    Mean F1 score for all  classifiers.
    """
    
    f1_score_list = []
    for i in range(np.shape(predicted_labels)[1]):
        f1 = f1_score(np.array(real_labels)[:, i], predicted_labels[:, i], average='micro')
        f1_score_list.append(f1)
        
    f1_score_avg = np.mean(f1_score_list)
    return f1_score_avg

In [32]:
# Apply grid search
parameters = {'vect__min_df': [5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[60],
              'clf__estimator__max_depth': [10, 15],
              'clf__estimator__n_jobs': [-1]}

scorer = make_scorer(define_mean_f1_score)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, n_jobs=-1, verbose = 48)

# Find best parameters
model_tuned = cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=5, score=0.9310260061709559, total=  37.4s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   54.5s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=5, score=0.930979384938799, total=  37.4s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=True, vect__min_df=5, score=0.9302885430441121, total=  37.4s
[CV] clf__estimator__max_depth=10, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=False, vect__min_df=5 


...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  9.4min remaining:    0.0s


[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=False, vect__min_df=5, score=0.9339249991523412, total=  42.3s
[CV] clf__estimator__max_depth=15, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed: 10.4min remaining:    0.0s


[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=60, clf__estimator__n_jobs=-1, tfidf__use_idf=False, vect__min_df=5, score=0.9330773403858542, total=  42.6s


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 11.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 11.4min finished


In [33]:
# Grid search results
model_tuned.cv_results_

{'mean_fit_time': array([ 28.69537592,  28.21673004,  34.02817504,  33.45290812]),
 'std_fit_time': array([ 0.06699117,  0.09894574,  0.30636664,  0.22320548]),
 'mean_score_time': array([ 8.66879956,  8.69653988,  8.91009474,  8.90812445]),
 'std_score_time': array([ 0.07284323,  0.03052165,  0.0250247 ,  0.06464985]),
 'param_clf__estimator__max_depth': masked_array(data = [10 10 15 15],
              mask = [False False False False],
        fill_value = ?),
 'param_clf__estimator__n_estimators': masked_array(data = [60 60 60 60],
              mask = [False False False False],
        fill_value = ?),
 'param_clf__estimator__n_jobs': masked_array(data = [-1 -1 -1 -1],
              mask = [False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True False True False],
              mask = [False False False False],
        fill_value = ?),
 'param_vect__min_df': masked_array(data = [5 5 5 5],
              mask = [False False False False],
 

In [34]:
# Mean score for the best model
np.max(model_tuned.cv_results_['mean_test_score'])

0.93347856553532471

In [35]:
# Parameters for the best model
model_tuned.best_params_

{'clf__estimator__max_depth': 15,
 'clf__estimator__n_estimators': 60,
 'clf__estimator__n_jobs': -1,
 'tfidf__use_idf': False,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
# Evaluation metrics for test set
tuned_pred_test = model_tuned.predict(X_test)

eval_metrics_tuned = eval_metrics(np.array(y_test), tuned_pred_test, col_names)

In [37]:
eval_metrics_tuned

,Accuracy,Precision,Recall,F1
related,0.766860,0.766860,0.766860,0.766860
request,0.826213,0.826213,0.826213,0.826213
offer,0.994660,0.994660,0.994660,0.994660
aid_related,0.708727,0.708727,0.708727,0.708727
medical_help,0.919896,0.919896,0.919896,0.919896
medical_products,0.946750,0.946750,0.946750,0.946750
search_and_rescue,0.970400,0.970400,0.970400,0.970400
security,0.981538,0.981538,0.981538,0.981538
military,0.966738,0.966738,0.966738,0.966738
child_alone,1.000000,1.000000,1.000000,1.000000


In [38]:
eval_metrics_tuned.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.929928,0.929928,0.929928,0.929928
std,0.073431,0.073431,0.073431,0.073431
min,0.708727,0.708727,0.708727,0.708727
25%,0.911771,0.911771,0.911771,0.911771
50%,0.950793,0.950793,0.950793,0.950793
75%,0.982530,0.982530,0.982530,0.982530
max,1.000000,1.000000,1.000000,1.000000


In [39]:
# Print mean f1 score
print("F1 score mean: ", eval_metrics_tuned['F1'].mean())

F1 score mean:  0.929928288068


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [53]:
%%time
# Testing a pure decision tree classifier
moc = MultiOutputClassifier(DecisionTreeClassifier())

pipeline_moc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 777)
pipeline_moc.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred_moc = pipeline_moc.predict(X_test)

CPU times: user 6min 2s, sys: 800 ms, total: 6min 3s
Wall time: 6min 5s


In [54]:
eval_metrics_moc = eval_metrics(np.array(y_test), y_pred_moc, col_names)

In [55]:
eval_metrics_moc

,Accuracy,Precision,Recall,F1
related,0.777846,0.777846,0.777846,0.777846
request,0.852609,0.852609,0.852609,0.852609
offer,0.991303,0.991303,0.991303,0.991303
aid_related,0.709643,0.709643,0.709643,0.709643
medical_help,0.906164,0.906164,0.906164,0.906164
medical_products,0.940037,0.940037,0.940037,0.940037
search_and_rescue,0.962313,0.962313,0.962313,0.962313
security,0.970400,0.970400,0.970400,0.970400
military,0.963229,0.963229,0.963229,0.963229
child_alone,1.000000,1.000000,1.000000,1.000000


In [56]:
eval_metrics_moc.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.933183,0.933183,0.933183,0.933183
std,0.066522,0.066522,0.066522,0.066522
min,0.709643,0.709643,0.709643,0.709643
25%,0.926800,0.926800,0.926800,0.926800
50%,0.954837,0.954837,0.954837,0.954837
75%,0.979936,0.979936,0.979936,0.979936
max,1.000000,1.000000,1.000000,1.000000


In [57]:
# Print mean f1 score
print("F1 score mean: ", eval_metrics_moc['F1'].mean())

F1 score mean:  0.933183297732


### 9. Export your model as a pickle file

Let us save tuned model.

In [40]:
pickle.dump(model_tuned, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.